In [ ]:
import numpy as np
import pandas as pd

from BE import BE_model as BE1
from BE_V2 import BE_model as BE2
from BE_Generative import BE_model as BE1_gen
from BE_V2_Generative import BE_model as BE2_gen
from Fitter import total_psychometric, post_correct_update_matrix
from Sampling import Asym_right, Asym_left, Uniform
from scipy.stats import uniform
from BE import Noise_generator, Delta_repulsion
import matplotlib.pyplot as plt
from BE_old import BE_model as BE_old
from BE_Generative_old import BE_model as BE_gen_old
from BE_V3 import BE_model as BE3
from BE_V3_Generative import BE_model as BE3_gen

**Version 1**:

BE model that uses $\eta_{learning}$ but not $\eta_{relax}$

In [ ]:
# Model parameters
sigma_noise = 0.3
A_repulsion = 0.7
eta_learning =0.6
sigma_boundary = 7

# Seed for reproducibility
seed= 40
burn_in_seed = 100


# Number of samples drawn from each distribution
n_samples = 5000

# Define stimulus space
max_range = 1+6*sigma_noise+2*A_repulsion*(1+6*sigma_noise)
min_range = -1-6*sigma_noise-2*A_repulsion*(1+6*sigma_noise)
num_points = round((max_range - min_range) * 1000)
x = np.linspace(min_range,max_range , num_points)

# Initial belief of the boundary location
y = uniform.pdf(x, loc=min_range, scale=max_range-min_range)

# Sampled stimuli from each distribution
s_uniform = Uniform(n_samples, seed)
s_AsymRight = Asym_right(n_samples, seed)
s_AsymLeft = Asym_left(n_samples, seed)


# Perceived Stimuli
# Uniform
s_tildeU = s_uniform + Noise_generator(len(s_uniform), seed, sigma_noise)
s_hat_uniform = Delta_repulsion(A_repulsion, s_tildeU)
categories_uniform = np.where(s_uniform > 0, 1, 0)

# Asym-right
s_tildeR = s_AsymRight + Noise_generator(len(s_AsymRight), seed, sigma_noise)
s_hat_AsymRight = Delta_repulsion(A_repulsion, s_tildeR)
categories_AsymRight = np.where(s_AsymRight > 0, 1, 0)

# Asym_left
s_tildeL = s_AsymLeft + Noise_generator(len(s_AsymLeft), seed, sigma_noise)
s_hat_AsymLeft = Delta_repulsion(A_repulsion, s_tildeL)
categories_AsymLeft = np.where(s_AsymLeft > 0, 1, 0)

# No response
No_resp = np.full(n_samples, False)

# Set burn-in mode
mode = 'simulated'

Not_Blockstart = np.full(n_samples, True)
Not_Blockstart[0] = False

# Generate choices for BE agent under uniform distributions with given hyperparameters
chooseB_uniform, reward_uniform = BE1_gen(x, y, s_hat_uniform, categories_uniform, sigma_noise, A_repulsion, eta_learning, sigma_boundary,
                                           No_resp, seed, burn_in_seed, mode)


# Generate choices for BE agent under Asym-right distributions with given hyperparameters
chooseB_AsymRight, reward_AsymRight = BE1_gen(x, y, s_hat_AsymRight, categories_AsymRight, sigma_noise, A_repulsion, eta_learning, sigma_boundary, 
                                              No_resp, seed, burn_in_seed, mode)


# Generate choices for BE agent under Asym-left distributions with given hyperparameters
chooseB_AsymLeft, reward_AsymLeft = BE1_gen(x, y, s_hat_AsymLeft,categories_AsymLeft , sigma_noise, A_repulsion, eta_learning, sigma_boundary,
                                             No_resp, seed, burn_in_seed, mode)

y_Uniform, _ = total_psychometric(s_uniform, chooseB_uniform, No_resp)
y_HardA, _ = total_psychometric(s_AsymRight, chooseB_AsymRight, No_resp)
y_HardB, _ = total_psychometric(s_AsymLeft, chooseB_AsymLeft, No_resp)
x_plot = np.linspace(-1, 1, 100000)
plt.plot(x_plot, y_Uniform)
plt.plot(x_plot, y_HardA)
plt.plot(x_plot, y_HardB)
plt.xlim([-1,1])
plt.ylim([0, 1])
plt.show()


In [ ]:
np.average(reward_uniform)

In [ ]:
# Generate choices for BE agent under uniform distributions with given hyperparameters
update_matrix_uniform, update_matrix_uniform = BE1(post_correct_update_matrix, x, y, s_uniform, s_hat_uniform, categories_uniform, sigma_noise, A_repulsion, eta_learning, sigma_boundary,
             No_resp, Not_Blockstart, seed, mode)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import uniform

# ==========================================
# 1. Folder Creation & Parameter Setup
# ==========================================
# Create the directory for saving PDFs
output_folder = "Parameter_Sweep_Results"
os.makedirs(output_folder, exist_ok=True)

# Outer loop ranges (PDF generation)
sigma_noise_vals = [0.1, 0.3, 0.5]  
A_repulsion_vals = [0.3, 0.7, 1.0]  

# Inner 10x10 grid ranges
eta_learning_vals = np.linspace(0.1, 1.0, 10)     
sigma_boundary_vals = np.linspace(1.0, 10.0, 10)  

# ==========================================
# 2. Global Constants & Pre-computations
# ==========================================
seed = 40
burn_in_seed = 100
n_samples = 5000
mode = 'simulated'

# Pre-generate stimuli
s_uniform = Uniform(n_samples, seed)
s_AsymRight = Asym_right(n_samples, seed)
s_AsymLeft = Asym_left(n_samples, seed)

categories_uniform = np.where(s_uniform > 0, 1, 0)
categories_AsymRight = np.where(s_AsymRight > 0, 1, 0)
categories_AsymLeft = np.where(s_AsymLeft > 0, 1, 0)

No_resp = np.full(n_samples, False)
Not_Blockstart = np.full(n_samples, True)
Not_Blockstart[0] = False

x_plot = np.linspace(-1, 1, 100000)

colors = {
    'Uniform': '#2F9A76',
    'Asym_left': '#007DC7',
    'Asym_right': '#C23F63'
}

# ==========================================
# 3. Execution of the Parameter Sweep
# ==========================================
for sigma_noise in sigma_noise_vals:
    for A_repulsion in A_repulsion_vals:
        
        # Save inside the newly created folder
        filename = os.path.join(output_folder, f"Sweep_Noise_{sigma_noise:.2f}_Repulsion_{A_repulsion:.2f}.pdf")
        print(f"Generating: {filename} ...")
        
        with PdfPages(filename) as pdf:
            fig, axes = plt.subplots(10, 10, figsize=(50, 40), dpi=300)
            
            # --- Update Space & Perceived Stimuli ---
            max_range = 1 + 6*sigma_noise + 2*A_repulsion*(1+6*sigma_noise)
            min_range = -1 - 6*sigma_noise - 2*A_repulsion*(1+6*sigma_noise)
            num_points = round((max_range - min_range) * 1000)
            x = np.linspace(min_range, max_range, num_points)
            y = uniform.pdf(x, loc=min_range, scale=max_range-min_range)

            s_tildeU = s_uniform + Noise_generator(len(s_uniform), seed, sigma_noise)
            s_hat_uniform = Delta_repulsion(A_repulsion, s_tildeU)

            s_tildeR = s_AsymRight + Noise_generator(len(s_AsymRight), seed, sigma_noise)
            s_hat_AsymRight = Delta_repulsion(A_repulsion, s_tildeR)

            s_tildeL = s_AsymLeft + Noise_generator(len(s_AsymLeft), seed, sigma_noise)
            s_hat_AsymLeft = Delta_repulsion(A_repulsion, s_tildeL)

            # --- Inner 10x10 Loop ---
            for i, eta_learning in enumerate(eta_learning_vals):
                for j, sigma_boundary in enumerate(sigma_boundary_vals):
                    ax = axes[i, j]
                    
                    # 1. Generate choices AND capture rewards
                    chooseB_uniform, reward_uniform = BE1_gen(
                        x, y, s_hat_uniform, categories_uniform, sigma_noise, A_repulsion, 
                        eta_learning, sigma_boundary, No_resp, seed, burn_in_seed, mode
                    )

                    chooseB_AsymRight, reward_AsymRight = BE1_gen(
                        x, y, s_hat_AsymRight, categories_AsymRight, sigma_noise, A_repulsion, 
                        eta_learning, sigma_boundary, No_resp, seed, burn_in_seed, mode
                    )

                    chooseB_AsymLeft, reward_AsymLeft = BE1_gen(
                        x, y, s_hat_AsymLeft, categories_AsymLeft, sigma_noise, A_repulsion, 
                        eta_learning, sigma_boundary, No_resp, seed, burn_in_seed, mode
                    )

                    # 2. Calculate psychometric curves
                    y_Uniform, _ = total_psychometric(s_uniform, chooseB_uniform, No_resp)
                    y_HardA, _ = total_psychometric(s_AsymRight, chooseB_AsymRight, No_resp)
                    y_HardB, _ = total_psychometric(s_AsymLeft, chooseB_AsymLeft, No_resp)   
                    
                    # 3. Plot curves
                    ax.plot(x_plot, y_Uniform, color=colors['Uniform'])
                    ax.plot(x_plot, y_HardB, color=colors['Asym_left']) 
                    ax.plot(x_plot, y_HardA, color=colors['Asym_right'])
                    
                    # 4. Calculate Mean Rewards
                    mean_rew_U = np.mean(reward_uniform)
                    mean_rew_R = np.mean(reward_AsymRight)
                    mean_rew_L = np.mean(reward_AsymLeft)
                    
                    # 5. Add Text with Narrow Font
                    # transform=ax.transAxes keeps the text in the top-left (0.05, 0.95) of the axes box
                    text_str = f"Avg Reward\nUni: {mean_rew_U:.3f}\nL: {mean_rew_L:.3f}\nR: {mean_rew_R:.3f}"
                    ax.text(0.05, 0.95, text_str, transform=ax.transAxes, 
                            verticalalignment='top', fontsize=9, 
                            fontstretch='condensed', weight='light', color='#333333')
                    
                    # 6. Formatting the subplot
                    ax.set_xlim([-1, 1])
                    ax.set_ylim([0, 1])
                    ax.set_title(f"eta={eta_learning:.2f}, sig_b={sigma_boundary:.2f}", fontsize=10)
                    
                    if j > 0: ax.set_yticklabels([])
                    if i < 9: ax.set_xticklabels([])

            # Save the 10x10 figure into the PDF
            fig.tight_layout()
            pdf.savefig(fig)
            plt.close(fig) 

print("Parameter sweep complete! Check the 'Parameter_Sweep_Results' folder.")

**Version 2**

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import uniform

# ==========================================
# 1. Folder Creation & Parameter Setup
# ==========================================
# Create the directory for saving PDFs
output_folder = "V22-Parameter_Sweep_Results"
os.makedirs(output_folder, exist_ok=True)

# Outer loop ranges (PDF generation)
sigma_noise_vals = [0.1, 0.3, 0.5]  
A_repulsion_vals = [0.0, 0.15]  

# Inner 10x10 grid ranges
eta_learning_vals = np.linspace(0.5, 10, 10)      # e.g., 1 to 100
sigma_boundary_vals = np.linspace(10, 30, 10)    # e.g., 1 to 100

# ==========================================
# 2. Global Constants & Pre-computations
# ==========================================
seed = 40
burn_in_seed = 100
n_samples = 5000
mode = 'simulated'

# Pre-generate stimuli
s_uniform = Uniform(n_samples, seed)
s_AsymRight = Asym_right(n_samples, seed)
s_AsymLeft = Asym_left(n_samples, seed)

categories_uniform = np.where(s_uniform > 0, 1, 0)
categories_AsymRight = np.where(s_AsymRight > 0, 1, 0)
categories_AsymLeft = np.where(s_AsymLeft > 0, 1, 0)

No_resp = np.full(n_samples, False)
Not_Blockstart = np.full(n_samples, True)
Not_Blockstart[0] = False

x_plot = np.linspace(-1, 1, 100000)

colors = {
    'Uniform': '#2F9A76',
    'Asym_left': '#007DC7',
    'Asym_right': '#C23F63'
}

# ==========================================
# 3. Execution of the Parameter Sweep
# ==========================================
for sigma_noise in sigma_noise_vals:
    for A_repulsion in A_repulsion_vals:
        
        # Save inside the newly created folder
        filename = os.path.join(output_folder, f"V22-Sweep_Noise_{sigma_noise:.2f}_Repulsion_{A_repulsion:.2f}.pdf")
        print(f"Generating: {filename} ...")
        
        with PdfPages(filename) as pdf:
            fig, axes = plt.subplots(10, 10, figsize=(50, 40), dpi=300)
            
            # --- Update Space & Perceived Stimuli ---
            max_range = 1 + 6*sigma_noise + 2*A_repulsion*(1+6*sigma_noise)
            min_range = -1 - 6*sigma_noise - 2*A_repulsion*(1+6*sigma_noise)
            num_points = round((max_range - min_range) * 1000)
            x = np.linspace(min_range, max_range, num_points)
            y = uniform.pdf(x, loc=min_range, scale=max_range-min_range)

            s_tildeU = s_uniform + Noise_generator(len(s_uniform), seed, sigma_noise)
            s_hat_uniform = Delta_repulsion(A_repulsion, s_tildeU)

            s_tildeR = s_AsymRight + Noise_generator(len(s_AsymRight), seed, sigma_noise)
            s_hat_AsymRight = Delta_repulsion(A_repulsion, s_tildeR)

            s_tildeL = s_AsymLeft + Noise_generator(len(s_AsymLeft), seed, sigma_noise)
            s_hat_AsymLeft = Delta_repulsion(A_repulsion, s_tildeL)

            # --- Inner 10x10 Loop ---
            for i, eta_learning in enumerate(eta_learning_vals):
                for j, sigma_boundary in enumerate(sigma_boundary_vals):
                    ax = axes[i, j]
                    
                    # 1. Generate choices AND capture rewards
                    chooseB_uniform, reward_uniform = BE2_gen(
                        x, y, s_hat_uniform, categories_uniform, sigma_noise, A_repulsion, 
                        eta_learning, sigma_boundary, No_resp, seed, burn_in_seed, mode
                    )

                    chooseB_AsymRight, reward_AsymRight = BE2_gen(
                        x, y, s_hat_AsymRight, categories_AsymRight, sigma_noise, A_repulsion, 
                        eta_learning, sigma_boundary, No_resp, seed, burn_in_seed, mode
                    )

                    chooseB_AsymLeft, reward_AsymLeft = BE2_gen(
                        x, y, s_hat_AsymLeft, categories_AsymLeft, sigma_noise, A_repulsion, 
                        eta_learning, sigma_boundary, No_resp, seed, burn_in_seed, mode
                    )

                    # 2. Calculate psychometric curves
                    y_Uniform, _ = total_psychometric(s_uniform, chooseB_uniform, No_resp)
                    y_HardA, _ = total_psychometric(s_AsymRight, chooseB_AsymRight, No_resp)
                    y_HardB, _ = total_psychometric(s_AsymLeft, chooseB_AsymLeft, No_resp)   
                    
                    # 3. Plot curves
                    ax.plot(x_plot, y_Uniform, color=colors['Uniform'])
                    ax.plot(x_plot, y_HardB, color=colors['Asym_left']) 
                    ax.plot(x_plot, y_HardA, color=colors['Asym_right'])
                    
                    # 4. Calculate Mean Rewards
                    mean_rew_U = np.mean(reward_uniform)
                    mean_rew_R = np.mean(reward_AsymRight)
                    mean_rew_L = np.mean(reward_AsymLeft)
                    
                    # 5. Add Text with Narrow Font
                    # transform=ax.transAxes keeps the text in the top-left (0.05, 0.95) of the axes box
                    text_str = f"Avg Reward\nUni: {mean_rew_U:.3f}\nL: {mean_rew_L:.3f}\nR: {mean_rew_R:.3f}"
                    ax.text(0.05, 0.95, text_str, transform=ax.transAxes, 
                            verticalalignment='top', fontsize=9, 
                            fontstretch='condensed', weight='light', color='#333333')
                    
                    # 6. Formatting the subplot
                    ax.set_xlim([-1, 1])
                    ax.set_ylim([0, 1])
                    ax.set_title(f"eta={eta_learning:.2f}, sig_b={sigma_boundary:.2f}", fontsize=10)
                    
                    if j > 0: ax.set_yticklabels([])
                    if i < 9: ax.set_xticklabels([])

            # Save the 10x10 figure into the PDF
            fig.tight_layout()
            pdf.savefig(fig)
            plt.close(fig) 

print("V22-Parameter sweep complete! Check the 'V22-Parameter_Sweep_Results' folder.")

**V3**

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import uniform

# ==========================================
# 1. Folder Creation & Parameter Setup
# ==========================================
# Create the directory for saving PDFs
output_folder = "V3-Parameter_Sweep_Results"
os.makedirs(output_folder, exist_ok=True)

# Outer loop ranges (PDF generation)
sigma_noise_vals = [0.3, 0.5]  
A_repulsion_vals = [0.0, 0.15]  

# Inner 10x10 grid ranges
eta_learning_vals = np.linspace(0.1, 1.0, 10)     
sigma_boundary_vals = np.linspace(1.0, 10.0, 10)  

# ==========================================
# 2. Global Constants & Pre-computations
# ==========================================
seed = 40
burn_in_seed = 100
n_samples = 5000
mode = 'simulated'

# Pre-generate stimuli
s_uniform = Uniform(n_samples, seed)
s_AsymRight = Asym_right(n_samples, seed)
s_AsymLeft = Asym_left(n_samples, seed)

categories_uniform = np.where(s_uniform > 0, 1, 0)
categories_AsymRight = np.where(s_AsymRight > 0, 1, 0)
categories_AsymLeft = np.where(s_AsymLeft > 0, 1, 0)

No_resp = np.full(n_samples, False)
Not_Blockstart = np.full(n_samples, True)
Not_Blockstart[0] = False

x_plot = np.linspace(-1, 1, 100000)

colors = {
    'Uniform': '#2F9A76',
    'Asym_left': '#007DC7',
    'Asym_right': '#C23F63'
}

# ==========================================
# 3. Execution of the Parameter Sweep
# ==========================================
for sigma_noise in sigma_noise_vals:
    for A_repulsion in A_repulsion_vals:
        
        # Save inside the newly created folder
        filename = os.path.join(output_folder, f"V3-Sweep_Noise_{sigma_noise:.2f}_Repulsion_{A_repulsion:.2f}.pdf")
        print(f"Generating: {filename} ...")
        
        with PdfPages(filename) as pdf:
            fig, axes = plt.subplots(10, 10, figsize=(50, 40), dpi=300)
            
            # --- Update Space & Perceived Stimuli ---
            max_range = 1 + 6*sigma_noise + 2*A_repulsion*(1+6*sigma_noise)
            min_range = -1 - 6*sigma_noise - 2*A_repulsion*(1+6*sigma_noise)
            num_points = round((max_range - min_range) * 1000)
            x = np.linspace(min_range, max_range, num_points)
            y = uniform.pdf(x, loc=min_range, scale=max_range-min_range)

            s_tildeU = s_uniform + Noise_generator(len(s_uniform), seed, sigma_noise)
            s_hat_uniform = Delta_repulsion(A_repulsion, s_tildeU)

            s_tildeR = s_AsymRight + Noise_generator(len(s_AsymRight), seed, sigma_noise)
            s_hat_AsymRight = Delta_repulsion(A_repulsion, s_tildeR)

            s_tildeL = s_AsymLeft + Noise_generator(len(s_AsymLeft), seed, sigma_noise)
            s_hat_AsymLeft = Delta_repulsion(A_repulsion, s_tildeL)

            # --- Inner 10x10 Loop ---
            for i, eta_learning in enumerate(eta_learning_vals):
                for j, sigma_boundary in enumerate(sigma_boundary_vals):
                    ax = axes[i, j]
                    
                    # 1. Generate choices AND capture rewards
                    chooseB_uniform, reward_uniform = BE3_gen(
                        x, y, s_hat_uniform, categories_uniform, sigma_noise, A_repulsion, 
                        eta_learning, sigma_boundary, No_resp, seed, burn_in_seed, mode
                    )

                    chooseB_AsymRight, reward_AsymRight = BE3_gen(
                        x, y, s_hat_AsymRight, categories_AsymRight, sigma_noise, A_repulsion, 
                        eta_learning, sigma_boundary, No_resp, seed, burn_in_seed, mode
                    )

                    chooseB_AsymLeft, reward_AsymLeft = BE3_gen(
                        x, y, s_hat_AsymLeft, categories_AsymLeft, sigma_noise, A_repulsion, 
                        eta_learning, sigma_boundary, No_resp, seed, burn_in_seed, mode
                    )

                    # 2. Calculate psychometric curves
                    y_Uniform, _ = total_psychometric(s_uniform, chooseB_uniform, No_resp)
                    y_HardA, _ = total_psychometric(s_AsymRight, chooseB_AsymRight, No_resp)
                    y_HardB, _ = total_psychometric(s_AsymLeft, chooseB_AsymLeft, No_resp)   
                    
                    # 3. Plot curves
                    ax.plot(x_plot, y_Uniform, color=colors['Uniform'])
                    ax.plot(x_plot, y_HardB, color=colors['Asym_left']) 
                    ax.plot(x_plot, y_HardA, color=colors['Asym_right'])
                    
                    # 4. Calculate Mean Rewards
                    mean_rew_U = np.mean(reward_uniform)
                    mean_rew_R = np.mean(reward_AsymRight)
                    mean_rew_L = np.mean(reward_AsymLeft)
                    
                    # 5. Add Text with Narrow Font
                    # transform=ax.transAxes keeps the text in the top-left (0.05, 0.95) of the axes box
                    text_str = f"Avg Reward\nUni: {mean_rew_U:.3f}\nL: {mean_rew_L:.3f}\nR: {mean_rew_R:.3f}"
                    ax.text(0.05, 0.95, text_str, transform=ax.transAxes, 
                            verticalalignment='top', fontsize=9, 
                            fontstretch='condensed', weight='light', color='#333333')
                    
                    # 6. Formatting the subplot
                    ax.set_xlim([-1, 1])
                    ax.set_ylim([0, 1])
                    ax.set_title(f"eta={eta_learning:.2f}, sig_b={sigma_boundary:.2f}", fontsize=10)
                    
                    if j > 0: ax.set_yticklabels([])
                    if i < 9: ax.set_xticklabels([])

            # Save the 10x10 figure into the PDF
            fig.tight_layout()
            pdf.savefig(fig)
            plt.close(fig) 

print("V3-Parameter sweep complete! Check the 'V3-Parameter_Sweep_Results' folder.")